In [1]:
from __future__ import division
import numpy as np
import pandas as pd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
categories = {0: 'pelota', 1: 'perro', 2: 'taza', 3: 'tractor'}
#rcategories = {v: k for (k, v) in categories}
rcategories = {'pelota': 0, 'perro': 1, 'taza': 2, 'tractor': 3}

In [3]:

data = pd.read_csv('../data/summary_data.csv')

In [4]:
data.describe()

,item,pred_combined,pred_rgb,pred_pcloud,entropy_rgb,entropy_pcloud,margin_rgb,margin_pcloud,jsd
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,28.000000,34.000000,38.000000
mean,18.500000,1.526316,1.657895,1.210526,0.654488,1.250784,16.571429,8.823529,0.467272
std,11.113055,1.032887,0.937977,1.118829,0.585092,0.550188,8.288451,5.900542,0.211509
min,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,1.000000,1.000000,0.003852
25%,9.250000,1.000000,1.000000,0.000000,0.052711,1.062525,9.750000,4.000000,0.328821
50%,18.500000,1.000000,1.500000,1.000000,0.559509,1.289038,16.000000,7.000000,0.462371
75%,27.750000,2.000000,2.000000,2.000000,1.094880,1.585963,26.000000,14.000000,0.627871
max,37.000000,3.000000,3.000000,3.000000,1.879936,2.004310,28.000000,21.000000,1.000000


In [5]:
data.ix[:, 1:3].head()

,y,pred_combined
0,pelota,2
1,pelota,2
2,pelota,2
3,pelota,0
4,pelota,0


In [6]:
labels = data.y
data.y = data.y.apply(lambda x: rcategories[x])

In [7]:
data.ix[:, 1:3].T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
y,0,0,0,0,0,1,1,1,1,1,...,3,3,2,2,2,2,2,2,2,2
pred_combined,2,2,2,0,0,1,1,1,1,1,...,3,0,2,0,0,2,2,2,2,0


#  Metrics

## F1-Score and Accuracy

In [8]:
from sklearn.metrics import f1_score, accuracy_score

In [9]:
print "Accuracy Combined: \t", accuracy_score(data.y, data.pred_combined)
print "Accuracy RGB: \t\t", accuracy_score(data.y, data.pred_rgb)
print "Accuracy PCloud: \t", accuracy_score(data.y, data.pred_pcloud)

Accuracy Combined: 	0.815789473684
Accuracy RGB: 		0.763157894737
Accuracy PCloud: 	0.289473684211


In [10]:
print "F1 Combined: \t", f1_score(data.y, data.pred_combined)
print "F1 RGB: \t", f1_score(data.y, data.pred_rgb)
print "F1 PCloud: \t", f1_score(data.y, data.pred_pcloud)

F1 Combined: 	0.82347015865
F1 RGB: 	0.748269939138
F1 PCloud: 	0.256467741362


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_

## Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
cmat = confusion_matrix(data.y, data.pred_combined)
cmat

array([[ 2,  0,  3,  0],
       [ 0, 15,  0,  0],
       [ 3,  0,  5,  0],
       [ 1,  0,  0,  9]])

In [13]:
catnames = ['pelota', 'perro', 'taza', 'tractor']

In [14]:
pcm = pd.DataFrame(cmat, index=catnames, columns=catnames)
pcm

,pelota,perro,taza,tractor
pelota,2,0,3,0
perro,0,15,0,0
taza,3,0,5,0
tractor,1,0,0,9


##  Normalized Confusion Matrix

In [15]:
normalized = np.zeros([4,4])
for index, row in enumerate(cmat):
    normalized[index] = np.true_divide(row, sum(row))
normalized

array([[ 0.4  ,  0.   ,  0.6  ,  0.   ],
       [ 0.   ,  1.   ,  0.   ,  0.   ],
       [ 0.375,  0.   ,  0.625,  0.   ],
       [ 0.1  ,  0.   ,  0.   ,  0.9  ]])

In [16]:
cm_normalized = pd.DataFrame(normalized, index=catnames, columns=catnames)
cm_normalized = cm_normalized.apply(np.round, args=(2,))
cm_normalized

,pelota,perro,taza,tractor
pelota,0.40,0,0.60,0.0
perro,0.00,1,0.00,0.0
taza,0.38,0,0.62,0.0
tractor,0.10,0,0.00,0.9


In [17]:
cm_normalized.values.diagonal().mean()

0.72999999999999998

In [18]:
from sklearn.metrics import precision_score, recall_score